In [ ]:
import pandas as pd 
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
import logging
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)

In [7]:
file_list = drive.ListFile({'q': "'10Fs-dG_GyXIf0ZYKbmXcabvhFtayMbmt' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: More, id: 1LbKM_iESxb5rpTZLZAwnWASOCRMpfK7x
title: labels_with_bbox.json, id: 1oaXHHoTR0lea-b73DynERdM6z889TSa1
title: راهنمای استفاده از داده ها.pdf, id: 1FM-1vUvkcBzmXYvjV1cRDfVBUgVeduWr
title: labels.json, id: 1XVwbPFCHKHmgPTZPRRcAGS3WYXTIciCz
title: images.zip, id: 1b6bnYM6_qsJA2Upezl20E9Yk_crIKgCB
title: captions.json, id: 1dAyIrlkez3GsluFAddpuo4YTM9L7Xw0U
title: categories_info.json, id: 1LrSfF57Ia5iB_4o4tygCieg1wA_QRb4R
title: images_info.json, id: 17FIpIpvz7XsxV3FPtV5iNOGsdcfiSRP-


In [ ]:
data_downloaded = drive.CreateFile({'id': '1XVwbPFCHKHmgPTZPRRcAGS3WYXTIciCz'})
data_downloaded.GetContentFile('labels.json')
data_downloaded = drive.CreateFile({'id': '1b6bnYM6_qsJA2Upezl20E9Yk_crIKgCB'})
data_downloaded.GetContentFile('images.zip')
data_downloaded = drive.CreateFile({'id': '1dAyIrlkez3GsluFAddpuo4YTM9L7Xw0U'})
data_downloaded.GetContentFile('captions.json')
data_downloaded = drive.CreateFile({'id': '1LrSfF57Ia5iB_4o4tygCieg1wA_QRb4R'})
data_downloaded.GetContentFile('categories_info.json')
data_downloaded = drive.CreateFile({'id': '17FIpIpvz7XsxV3FPtV5iNOGsdcfiSRP-'})
data_downloaded.GetContentFile('images_info.json')
# data_downloaded = drive.CreateFile({'id': '1LbKM_iESxb5rpTZLZAwnWASOCRMpfK7x'})
# data_downloaded.GetContentFile('More/')

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')
path = '/content/gdrive/My Drive/More/MoreImages'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Create a ZipFile Object and load sample.zip in it
from zipfile import ZipFile
with ZipFile('images.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('images')
with ZipFile('/content/gdrive/My Drive/More/MoreImages.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('images')

In [ ]:
def mergefolders(root_src_dir, root_dst_dir):
    for src_dir, dirs, files in os.walk(root_src_dir):
        dst_dir = src_dir.replace(root_src_dir, root_dst_dir, 1)
        if not os.path.exists(dst_dir):
            os.makedirs(dst_dir)
        for file_ in files:
            src_file = os.path.join(src_dir, file_)
            dst_file = os.path.join(dst_dir, file_)
            if os.path.exists(dst_file):
                os.remove(dst_file)
            shutil.copy(src_file, dst_dir)

**Merging All Photos**

In [ ]:
import shutil
import os
mergefolders('/content/images/MoreImages','/content/images/val2017')

In [13]:
import sys, time, os, warnings 
import numpy as np 
import pandas as pd
jpgs = os.listdir('images/val2017')
print("The number of jpg flies in image: {}".format(len(jpgs)))


The number of jpg flies in image: 15000


In [ ]:
import json
import numpy as np 
import pandas as pd
with open('labels.json') as json_file: lab = json.load(json_file)
with open('captions.json') as json_file: cap = json.load(json_file)
with open('/content/gdrive/My Drive/More/additional_labels.json') as json_file: morelab = json.load(json_file)
with open('/content/gdrive/My Drive/More/additional_captions.json') as json_file: morecap = json.load(json_file)
c=pd.DataFrame(lab+morelab)
c.pop('bbox')
f={}
for name in jpgs: 
  if (c['image_id']==int(name[0:12])).any():
    f.update({str(int(name[0:12])):c.loc[c['image_id']==int(name[0:12]),'category_id'].unique()})

In [15]:
# c.pop('bbox')
len(f.keys())

14867

In [16]:
len(jpgs)

15000

In [17]:
import json
with open('captions.json') as json_file: cap = json.load(json_file)
with open('/content/gdrive/My Drive/More/additional_captions.json') as json_file: morecap = json.load(json_file)
cap=pd.DataFrame(cap+morecap)
cap[cap['image_id']==179765]['caption']

0     A black Honda motorcycle parked in front of a ...
1         A Honda motorcycle parked in a grass driveway
5     A black Honda motorcycle with a dark burgundy ...
44    Ma motorcycle parked on the gravel in front of...
46    A motorcycle with its brake extended standing ...
Name: caption, dtype: object

**Creating dictionary of captions**

In [ ]:
descriptions=dict()
for name in jpgs: 
  v=list()
  for i in cap[cap['image_id']==int(name[0:12])]['caption']:
    v.append(i)
  descriptions.update({str(int(name[0:12])):v})

In [19]:

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
tf.enable_eager_execution()
# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt
# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle


In [20]:
len(descriptions.keys())

15000

# Storing captions and image names in vectors

In [ ]:
all_captions = []
all_img_name_vector = []
for key , values in descriptions.items():

  for ca in values:
    caption = '<start> ' + ca + ' <end>'
    image_id = int(key)
    full_coco_image_path = '/content/images/val2017'+'/' + '%012d.jpg' % (image_id)

    all_img_name_vector.append(full_coco_image_path)
    all_captions.append(caption)

train_captions, img_name_vector = shuffle(all_captions,
                                          all_img_name_vector,
                                          random_state=1)

# # Select the first 75043 captions from the shuffled set
train_captions = train_captions
img_name_vector = img_name_vector


In [22]:
len(train_captions)


75043

In [ ]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [24]:
image_model = tf.keras.applications.InceptionV3(include_top=False,weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output
image_features_extract_model = tf.keras.Model(new_input, hidden_layer)


87916544/87910968 [==============================] - 3s 0us/step


In [ ]:
# Get unique images
encode_train = sorted(set(img_name_vector))

image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

for img, path in image_dataset:
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())


In [ ]:
# maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)


In [ ]:
# 6000 popular  words are chosen from the vocabulary
top_k = 6000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)


In [ ]:
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'


In [ ]:
# Create the tokenized vectors
train_seqs = tokenizer.texts_to_sequences(train_captions)


**Padding**

In [ ]:
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')


In [ ]:
#  max_length  which is used to store the attention weights
max_length = calc_max_length(train_seqs)


# 80-20 split training and validation 

In [32]:
img_name_train, img_name_val, cap_train, cap_val = train_test_split(img_name_vector,
                                                                    cap_vector,
                                                                    test_size=0.2,
                                                                    random_state=0)
len(img_name_train), len(cap_train), len(img_name_val), len(cap_val)


(60034, 60034, 15009, 15009)

**Model hyperparameters**

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
vocab_size = top_k + 1
num_steps = len(img_name_train) // BATCH_SIZE
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 64


Load the numpy(feature) files

In [ ]:
def map_func(img_name, cap):
  img_tensor = np.load(img_name.decode('utf-8')+'.npy')
  return img_tensor, cap


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle and batch
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


**Attention model**

In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)
# features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)
# hidden shape == (batch_size, hidden_size)
# hidden_with_time_axis shape == (batch_size, 1, hidden_size)
  def call(self, features, hidden):

    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # score shape == (batch_size, 64, hidden_size)
    score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

    # attention_weights shape == (batch_size, 64, 1)
    attention_weights = tf.nn.softmax(self.V(score), axis=1)
    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights


**Encoder**

In [ ]:
class CNN_Encoder(tf.keras.Model):
    # This encoder passes extracted features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x


**Decoder**

In [ ]:
class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))


In [ ]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)


**Optimizer**

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)


In [ ]:
checkpoint_path = "./content/gdrive/My Drive/saving"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)


In [ ]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
  ckpt.restore(ckpt_manager.latest_checkpoint)


**Loss structure**

In [ ]:
loss_plot = []
@tf.function
def train_step(img_tensor, target):
  loss = 0

  # initializing the hidden state for each batch as captions are not related from image to image
  hidden = decoder.reset_state(batch_size=target.shape[0])

  dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

  with tf.GradientTape() as tape:
      features = encoder(img_tensor)

      for i in range(1, target.shape[1]):
          # piping extracted features through the decoder
          predictions, hidden, _ = decoder(dec_input, features, hidden)

          loss += loss_function(target[:, i], predictions)

          # teacher forcing
          dec_input = tf.expand_dims(target[:, i], 1)

  total_loss = (loss / int(target.shape[1]))

  trainable_variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradients, trainable_variables))

  return loss, total_loss


**Loss visulization and formulation**

In [ ]:
EPOCHS = 20

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
      ckpt_manager.save()

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1 Batch 0 Loss 1.9864
Epoch 1 Batch 100 Loss 1.0421
Epoch 1 Batch 200 Loss 0.9378
Epoch 1 Batch 300 Loss 0.9283
Epoch 1 Batch 400 Loss 0.8482
Epoch 1 Batch 500 Loss 0.8582
Epoch 1 Batch 600 Loss 0.7988
Epoch 1 Batch 700 Loss 0.7914
Epoch 1 Batch 800 Loss 0.7552
Epoch 1 Batch 900 Loss 0.6832
Epoch 1 Loss 0.870838
Time taken for 1 epoch 852.9357860088348 sec

Epoch 2 Batch 0 Loss 0.7483
Epoch 2 Batch 100 Loss 0.6864
Epoch 2 Batch 200 Loss 0.6812
Epoch 2 Batch 300 Loss 0.7460
Epoch 2 Batch 400 Loss 0.7131
Epoch 2 Batch 500 Loss 0.7368
Epoch 2 Batch 600 Loss 0.6966
Epoch 2 Batch 700 Loss 0.7071
Epoch 2 Batch 800 Loss 0.6787
Epoch 2 Batch 900 Loss 0.6107
Epoch 2 Loss 0.689028
Time taken for 1 epoch 1163.726729631424 sec

Epoch 3 Batch 0 Loss 0.6689
Epoch 3 Batch 100 Loss 0.6151
Epoch 3 Batch 200 Loss 0.6231
Epoch 3 Batch 300 Loss 0.6892
Epoch 3 Batch 400 Loss 0.6555
Epoch 3 Batch 500 Loss 0.6

In [ ]:
plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()


In [ ]:
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot


In [ ]:
def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result//2, len_result//2, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()


**# captions on the validation set**

In [ ]:

rid = np.random.randint(0, len(img_name_val))
image = img_name_val[rid]
real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate(image)

print ('Real Caption:', real_caption)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image, result, attention_plot)


In [ ]:
len(jpgs)

**Evaluation**

In [ ]:
result, attention_plot = evaluate(image_path)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image_path, result, attention_plot)
# opening the image
Image.open(image_path)
